In [38]:
market_rates = pd.read_excel('inputs/market_rates.xlsx')

In [21]:
market_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    260 non-null    datetime64[ns]
 1   ccy-1   260 non-null    float64       
 2   ccy-2   260 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 6.2 KB


In [39]:
market_rates = ( market_rates
                .set_index('date')
                .sort_index()
)
market_rates

,ccy-1,ccy-2
date,,
2018-11-14,1.149980,0.895255
2018-11-15,1.129599,0.905879
2018-11-16,1.126722,0.900414
2018-11-19,1.247645,0.910705
2018-11-20,1.123899,0.912825
...,...,...
2019-11-08,1.160726,0.877659
2019-11-11,1.166902,0.877539
2019-11-12,1.166031,0.883470


In [25]:
market_rates.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 260 entries, 2018-11-14 to 2019-11-14
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ccy-1   260 non-null    float16
 1   ccy-2   260 non-null    float16
dtypes: float16(2)
memory usage: 3.0 KB


In [43]:
df = pd.DataFrame(market_rates['ccy-1'])
df

,ccy-1
date,
2018-11-14,1.149980
2018-11-15,1.129599
2018-11-16,1.126722
2018-11-19,1.247645
2018-11-20,1.123899
...,...
2019-11-08,1.160726
2019-11-11,1.166902
2019-11-12,1.166031


In [99]:
import pandas as pd
import numpy as np
from typing import Dict, List

class Shift:
    @staticmethod
    def logarithmic(rates: pd.Series, 
                    n: int,
                    ) -> np.ndarray:
        shifted_rates = rates.shift(n).dropna()
        division_vector = rates[n:] / shifted_rates
        ln_vector = np.log(division_vector) * np.sqrt(n)
        return np.exp(ln_vector) - 1
    
    @staticmethod
    def absolute():
        ...

    @staticmethod
    def relative():
        ...
    
class MarketRates:
    def __init__(self, data: pd.DataFrame):
        self._data = data
    
    @classmethod
    def from_excel(cls, file_path: str):
        data = ( pd
                .read_excel(file_path)
                .set_index('date')
                .sort_index()
        )
        return cls(data)
    
    @property
    def data(self):
        return self._data
    
    
class Product:
    def __init__(self, 
                 product_name: str, 
                 spot_value: float,
                 market_rates: pd.DataFrame, 
                 n_periods: int,
                 ):
        self._name = product_name
        self._hist_data = market_rates[product_name].copy()
        self._spot_value = spot_value
        self._n_periods = n_periods
    
    @property
    def name(self):
        return self._name
    
    @property
    def pnl_vector(self) -> np.ndarray:
        log_shift_vector = Shift.logarithmic(self._hist_data, self._n_periods)
        pnl_vector = self._spot_value * log_shift_vector
        return pnl_vector
    
class ValueAtRisk:
    def __init__(self, 
                 weights: List[float],
                 products: List[Product],
                 ):
        self._weights = weights
        self._products = products

    @classmethod
    def with_weights(cls, 
                     scnd_weight: float, 
                     portf_values: Dict, 
                     rates: pd.DataFrame):
        products = [Product(key, value, rates) for key,value in portf_values.items()]
        weights = [scnd_weight, 1 - scnd_weight]
        return cls(weights, products)
    
    @property
    def total_pnl_vector(self) -> np.ndarray:
        pnl_vectors = [product.pnl_vector for product in self._products]
        return np.sum(pnl_vectors, axis=0)
    
    @property
    def value(self) -> float:
        total_pnl = self.total_pnl_vector
        worst_scenarios = [total_pnl.argpartition(-n)[-n] for n in (2,3)]
        display(worst_scenarios)
        value = np.dot(self._weights, worst_scenarios)
        return value

In [61]:
market_rates = MarketRates.from_excel('inputs/market_rates.xlsx')
market_rates.data

,ccy-1,ccy-2
date,,
2018-11-14,1.149980,0.895255
2018-11-15,1.129599,0.905879
2018-11-16,1.126722,0.900414
2018-11-19,1.247645,0.910705
2018-11-20,1.123899,0.912825
...,...,...
2019-11-08,1.160726,0.877659
2019-11-11,1.166902,0.877539
2019-11-12,1.166031,0.883470


In [62]:
portfolio_values = {
    "ccy-1": 153084.81,
    "ccy-2": 95891.51
}

In [100]:
products = [Product(key, value, market_rates.data, 1) for key,value in portfolio_values.items()]

In [101]:
value_at_risk = ValueAtRisk([0.4, 0,6], products)

In [102]:
portfolio_var = value_at_risk.value
portfolio_var

ValueError: shapes (3,) and (2,) not aligned: 3 (dim 0) != 2 (dim 0)

In [45]:
df['shifted_rate'] = df['ccy-1'].shift(1, fill_value=0)
df = df.iloc[1:].copy()
df['ln_shift'] = get_log_shift(df['shifted_rate'], df['ccy-1'])
df

,ccy-1,shifted_rate,ln_shift
date,,,
2018-11-15,1.129599,1.149980,-0.017723
2018-11-16,1.126722,1.129599,-0.002546
2018-11-19,1.247645,1.126722,0.107323
2018-11-20,1.123899,1.247645,-0.099184
2018-11-21,1.121730,1.123899,-0.001929
...,...,...,...
2019-11-08,1.160726,1.160712,0.000012
2019-11-11,1.166902,1.160726,0.005321
2019-11-12,1.166031,1.166902,-0.000746


In [46]:
spot_value = 153084.81

In [47]:
df['pnl'] = spot_value * df['ln_shift']
df

,ccy-1,shifted_rate,ln_shift,pnl
date,,,,
2018-11-15,1.129599,1.149980,-0.017723,-2713.184304
2018-11-16,1.126722,1.129599,-0.002546,-389.814058
2018-11-19,1.247645,1.126722,0.107323,16429.468702
2018-11-20,1.123899,1.247645,-0.099184,-15183.599917
2018-11-21,1.121730,1.123899,-0.001929,-295.358138
...,...,...,...,...
2019-11-08,1.160726,1.160712,0.000012,1.776895
2019-11-11,1.166902,1.160726,0.005321,814.575462
2019-11-12,1.166031,1.166902,-0.000746,-114.241063
